# Import Libraries

In [34]:
import argparse
import sqlite3
import pandas as pd
from tqdm import tqdm

# Import dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#def execute_query_pandas(path, query):
    #conn = sqlite3.connect(path)
    #df = pd.read_sql_query(query, conn)
    #return df

In [13]:
#source = "thenewyorktimes"
#query = "SELECT * FROM newsdata WHERE source='%s'" % source
#path = ("/content/drive/My Drive/nela-gt-2020.db")
#data = execute_query_pandas(path, query)

In [14]:
data

,id,date,source,title,content,author,url,published,published_utc,collection_utc
0,thenewyorktimes--2020-04-15--Trump Wanted a Ra...,2020-04-15,thenewyorktimes,"Trump Wanted a Radio Show, but He Didn’t Want ...",The White House declined to comment on Mr. Tru...,Elaina Plott,https://www.nytimes.com/2020/04/15/us/trump-ra...,"Wed, 15 Apr 2020 17:47:44 +0000",1586987264,1586997923
1,thenewyorktimes--2020-04-16--Elizabeth Warren ...,2020-04-16,thenewyorktimes,Elizabeth Warren Endorses Joe Biden for President,"Since Ms. Warren ended her own campaign, she a...",Maggie Astor and Katie Glueck,https://www.nytimes.com/2020/04/15/us/politics...,"Thu, 16 Apr 2020 00:35:16 +0000",1587011716,1586997925
2,thenewyorktimes--2020-04-14--‘Accelerate the E...,2020-04-14,thenewyorktimes,‘Accelerate the Endgame’: Obama’s Role in Wrap...,"Mr. Sanders, who in 2016 accused the Democrati...",Glenn Thrush,https://www.nytimes.com/2020/04/14/us/politics...,"Tue, 14 Apr 2020 20:31:13 +0000",1586910673,1586997927
3,"thenewyorktimes--2020-04-15--Hello, What’s Thi...",2020-04-15,thenewyorktimes,"Hello, What’s This? The Democrats Aren’t in Di...",WASHINGTON — It may have taken a once-in-a-cen...,Jonathan Martin,https://www.nytimes.com/2020/04/14/us/politics...,"Wed, 15 Apr 2020 15:16:02 +0000",1586978162,1586997923
4,thenewyorktimes--2020-04-16--Trump’s ‘Opening ...,2020-04-16,thenewyorktimes,Trump’s ‘Opening Our Country Council’ Runs Int...,The full membership of the group was unclear W...,"Annie Karni, Kate Kelly and David Gelles",https://www.nytimes.com/2020/04/15/us/politics...,"Thu, 16 Apr 2020 00:02:27 +0000",1587009747,1586997925
...,...,...,...,...,...,...,...,...,...,...
19800,thenewyorktimes--2020-04-04--Administration Tu...,2020-04-04,thenewyorktimes,Administration Turns to Private Firms to Help ...,WASHINGTON — The Small Business Administration...,Kenneth P. Vogel and David McCabe,https://www.nytimes.com/2020/04/03/us/politics...,"Sat, 04 Apr 2020 01:32:31 +0000",1585978351,1586002579
19801,thenewyorktimes--2020-04-04--Tensions Persist ...,2020-04-04,thenewyorktimes,Tensions Persist Between Trump and Medical Adv...,WASHINGTON — Rarely has the schism between Pre...,"Peter Baker, Maggie Haberman and James Glanz",https://www.nytimes.com/2020/04/03/us/politics...,"Sat, 04 Apr 2020 01:21:12 +0000",1585977672,1586002581
19802,thenewyorktimes--2020-04-04--A Debate Over Mas...,2020-04-04,thenewyorktimes,A Debate Over Masks Uncovers Deep White House ...,WASHINGTON — President Trump said on Friday th...,Michael D. Shear and Sheila Kaplan,https://www.nytimes.com/2020/04/03/us/politics...,"Sat, 04 Apr 2020 01:41:35 +0000",1585978895,1586002584
19803,thenewyorktimes--2020-04-04--The Navy Fired Ca...,2020-04-04,thenewyorktimes,The Navy Fired Captain Crozier After His Lette...,But a Navy official familiar with the situatio...,"Helene Cooper, Thomas Gibbons-Neff and Eric Sc...",https://www.nytimes.com/2020/04/03/us/politics...,"Sat, 04 Apr 2020 02:54:30 +0000",1585983270,1586002579


In [19]:
path = ("/content/drive/My Drive/nela-gt-2020.db")
conn = sqlite3.connect(path)

In [15]:
def execute_query(conn, query):
    # execute query on database and retrieve them with fetchall
    results = conn.cursor().execute(query).fetchall()
    return results

In [16]:
path = ("/content/drive/My Drive/labels.csv")
labels = dict()
with open(path) as fin:
    # Read out the header line from label file
    fin.readline()
    # Iterate over lines, taking the value from first column after name
    # i.e., aggregated label
    for line in fin:
        l = line.strip().split(",")
        source = l[0]
        if l[1] == "":  # NODATA for this entry, skip it
            continue
        labels[source] = int(l[1])  # get value from last column (label)

In [17]:
len(labels)

335

In [20]:
execute_query(conn, "SELECT * FROM sqlite_master WHERE type = 'table'")

[('table',
  'newsdata',
  'newsdata',
  2,
  'CREATE TABLE newsdata (id text primary key,date text,source text,title text,content text,author text,url text,published text,published_utc integer,collection_utc integer)'),
 ('table',
  'tweet',
  'tweet',
  4,
  'CREATE TABLE tweet (id text primary key,article_id text,embedded_tweet text)')]

In [21]:
total_no_sources = execute_query(conn, "SELECT distinct source FROM newsdata")
len(total_no_sources)

519

In [22]:
total_no_sources

[('21stcenturywire',),
 ('abcnews',),
 ('acnlatitudes',),
 ('activistpost',),
 ('adobochronicles',),
 ('adwnews',),
 ('ageofautism',),
 ('airwars',),
 ('aljazeera',),
 ('allianceadvancedhealth',),
 ('allianceforadvancedhealth',),
 ('alliancefornaturalhealth',),
 ('alliancefornaturalhealthbiasjunkscience',),
 ('alternativemediatelevisionamtv',),
 ('alternativemediatv',),
 ('althealthworks',),
 ('americablognews',),
 ('americanintelligencemedia',),
 ('anonymousconservative',),
 ('anti-impreialist',),
 ('asiapacificresearch',),
 ('australiannationalreview',),
 ('autisminvestigated',),
 ('awarenessact',),
 ('bbc',),
 ('bbcuk',),
 ('bearingarms',),
 ('beholdisrael',),
 ('bigleaguepolitics',),
 ('bipartisanreport',),
 ('birminghammail',),
 ('bonginoreport',),
 ('breaking911',),
 ('breitbart',),
 ('brightside',),
 ('businessinsider',),
 ('buzzfeed',),
 ('cbsnews',),
 ('channel4uk',),
 ('charlotteobserver',),
 ('chicagosun-times',),
 ('chicagotribune',),
 ('citizenfreepress',),
 ('civictribune

In [23]:
each_source_counts = execute_query(conn, "SELECT source, count(source) FROM newsdata group by source")

In [24]:
counts_dict = {}
for each_source in each_source_counts:
    if each_source[0] in labels:
        type_ = labels[each_source[0]]
        if type_ in counts_dict:
            counts_dict[type_] += each_source[1]
        else:
            counts_dict[type_] = each_source[1]
    else:
        print(each_source[0])
counts_dict

adobochronicles
alliancefornaturalhealth
alternativemediatv
americablognews
annielowrey
anonnews
anonymousconservative
anti-impreialist
antiimpreialist
anxiouschild
au
australiantimes
avn
azmirror
azpbs
ballinaadvocate
bbcuk
beat
bigrigs
bipartisanreport
bizstandardnews
bnnbloomberg
boisestatepublicradio
business
buzzfeednews
byronnews
cairnspost
caitlinflanagan
camelotdaily
campaignasia
cbssports
centraltelegraph
chicagosun-times
cityam
civictribune
coffscoastadvocate
collective-evolution
coloradopeakpolitics
confrontingraceinamerica
conservativehome
covid19business
covid19campaign
covid19news
cqnews
cronkitenews
csmonitor
dailyexaminer
dailymercury
defpen
delawareliberal
deneenborelli
dickmorrisblog
digiday
drudgereport
echonews
edyong
emarketer
environmentdepth
etonline
ezorigin
fakingnews
fivethirthyeight
fortruss
freedom-bunker
freedombunker
freedomoutpost
freemalaysiatoday
ftwestminsterblog
gattonstar
geelongadvertiser
gladstoneobserver
glossynews
greatgameindia
greenwichtime
gym

{0: 493140, 1: 262288, 2: 533335}

In [25]:
percent = 10
sample_sizes = []
for x in [0,2]:
    count = counts_dict[x]
    source_counts = [each for each in each_source_counts if each[0] in labels and labels[each[0]] == x]
    source_distribution = { each[0]: each[1]/count for each in source_counts}
    sampling_10 = int(count * (percent /100))
    sample_sizes += [(each , int(source_distribution[each] * sampling_10)) for each in source_distribution.keys() if int(source_distribution[each] * sampling_10)  != 0 ]

In [28]:
sampled_data = []
for src in tqdm(sample_sizes):
    res = execute_query(conn, "select * from newsdata where source ='" + src[0] + "' limit " + str(src[1]))
    sampled_data = sampled_data + res

100%|██████████| 182/182 [02:52<00:00,  1.06it/s]


In [38]:
sampled_df = pd.DataFrame(sampled_data, columns=["id" ,"date" ,"source" ,"title" ,"content" ,"author" ,"url","published" ,"published_utc" ,"collection_utc"])

In [39]:
sampled_df["source"].map(labels).value_counts()

2    53276
0    49278
Name: source, dtype: int64

In [40]:
sampled_df["Reliability"] = sampled_df["source"].map(labels)

In [41]:

sampled_df.sample(10)

,id,date,source,title,content,author,url,published,published_utc,collection_utc,Reliability
98262,thesun--2020-01-27--Kobe Bryant dead at 41: NB...,2020-01-27,thesun,Kobe Bryant dead at 41: NBA legend and daughte...,"KOBE Bryant and his oldest daughter, Gianna, h...",Sophia Caraballo Pineiro,https://www.thesun.co.uk/sport/10824514/kobe-b...,"Mon, 27 Jan 2020 21:23:32 +0000",1580178212,1580171201,2
9696,chicagotribune--2020-01-30--Prosecutors to pro...,2020-01-30,chicagotribune,Prosecutors to propose that Chicago Ald. Edwar...,Among the projects Burke tried to capitalize o...,Jason Meisner,http://www.chicagotribune.com/news/criminal-ju...,"Thu, 30 Jan 2020 13:26:54 PST",1580408814,1580428964,0
54897,citizenfreepress--2020-04-15--Flying lemur sna...,2020-04-15,citizenfreepress,Flying lemur snared in trap (disturbing)…,Post was not sent - check your email addresses...,Kane,https://www.citizenfreepress.com/breaking/what...,"Wed, 15 Apr 2020 02:34:35 +0000",1586932475,1586994323,2
72755,newyorkpost--2020-02-06--Rep. Paul Gosar troll...,2020-02-06,newyorkpost,Rep. Paul Gosar trolls Nancy Pelosi with claim...,Rep. Paul Gosar of Arizona went viral Thursday...,Steven Nelson,https://nypost.com/2020/02/06/rep-paul-gosar-t...,"Thu, 06 Feb 2020 19:09:11 -0500",1581034151,1581034508,2
41752,theseattletimes--2020-02-13--Thai mall reopens...,2020-02-13,theseattletimes,Thai mall reopens after honoring victims of ma...,"NAKHON RATCHASIMA, Thailand (AP) — The shoppin...",The Associated Press,https://www.seattletimes.com/nation-world/nati...,"Thu, 13 Feb 2020 03:41:23 -0800",1581583283,1581595368,0
15602,eveningstandard--2020-01-23--Soldier in Queen&...,2020-01-23,eveningstandard,Soldier in Queen&apos;s ceremonial regiment gu...,A soldier serving in the Queen's ceremonial re...,Rebecca Speare-Cole,https://www.standard.co.uk/news/crime/queen-so...,"Thu, 23 Jan 2020 19:50:51 GMT",1579827051,1579825723,0
70883,newsmax--2020-01-07--Bloomberg Spends $10 Mill...,2020-01-07,newsmax,Bloomberg Spends $10 Million on Super Bowl Ad,"Mike Bloomberg, former mayor of New York City ...",,https://www.newsmax.com/politics/mike-bloomber...,"Tue, 07 Jan 2020 15:09:51 EDT",1578427791,1578444861,2
92355,theindependent--2020-02-13--Stranger Things se...,2020-02-13,theindependent,Stranger Things season 4: Is Hopper actually d...,Stranger Things fans have noticed that the nam...,Jacob Stolworthy,https://www.independent.co.uk/arts-entertainme...,"Thu, 13 Feb 2020 08:56:35 GMT",1581602195,1581600919,2
20290,hotair--2020-02-09--First American death from ...,2020-02-09,hotair,"First American death from coronavirus, deaths ...",The first American citizen’s death from corona...,,https://hotair.com/archives/karen-townsend/202...,"Sun, 09 Feb 2020 17:01:20 Z",1581285680,1581288113,0
24841,nationalreview--2020-01-16--Andrew McCabe Prop...,2020-01-16,nationalreview,Andrew McCabe Proposes DOJ Accountability Offi...,Former acting FBI director Andy McCabe admitte...,Tobias Hoonhout,https://www.nationalreview.com/news/fisa-warra...,"Thu, 16 Jan 2020 22:44:45 +0000",1579232685,1579278769,0


In [42]:
sampled_df.to_csv("/content/drive/My Drive/nela10sample.csv", index=False)